In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn import svm

df=pd.read_csv('train.csv')
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [4]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:

df['Gender'].value_counts()


Gender
Male      489
Female    112
Name: count, dtype: int64

In [6]:

df['Self_Employed'].value_counts()


Self_Employed
No     500
Yes     82
Name: count, dtype: int64

In [7]:
df['LoanAmount'].value_counts()

LoanAmount
120.0    20
110.0    17
100.0    15
160.0    12
187.0    12
         ..
240.0     1
214.0     1
59.0      1
166.0     1
253.0     1
Name: count, Length: 203, dtype: int64

In [8]:

df['Dependents'].value_counts()


Dependents
0     345
1     102
2     101
3+     51
Name: count, dtype: int64

In [9]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [10]:
df['Gender'] = df['Gender'].fillna('Female')

In [11]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [12]:
df['Married'].value_counts()

Married
Yes    398
No     213
Name: count, dtype: int64

In [13]:
df.isnull().sum()

Loan_ID               0
Gender                0
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [14]:
df['Married'] = df['Married'].fillna('Yes')

In [15]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [16]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [17]:
df['Dependents'] = df['Dependents'].fillna('0')

In [18]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [19]:
df['Self_Employed'].value_counts()

Self_Employed
No     500
Yes     82
Name: count, dtype: int64

In [20]:
df['Self_Employed'] = df['Self_Employed'].fillna('No')

In [21]:
df['Self_Employed'].value_counts()

Self_Employed
No     532
Yes     82
Name: count, dtype: int64

In [22]:
Loan_Avg = np.mean(df['LoanAmount'])

In [23]:
df['LoanAmount'] = df['LoanAmount'].fillna(Loan_Avg )
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(Loan_Avg )

In [24]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [25]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.000000,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.000000,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.000000,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.000000,360.0,1.0,Urban,Y


In [26]:
df.sample(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
77,LP001259,Male,Yes,1,Graduate,Yes,1000,3022.0,110.0,360.0,1.0,Urban,N
119,LP001422,Female,No,0,Graduate,No,10408,0.0,259.0,360.0,1.0,Urban,Y
564,LP002832,Male,Yes,2,Graduate,No,8799,0.0,258.0,360.0,0.0,Urban,N
157,LP001543,Male,Yes,1,Graduate,No,9538,0.0,187.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [27]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [28]:
# label encoding
df.replace({"Gender":{'Female':0,'Male':1}},inplace=True)
# label encoding
df.replace({"Married":{'No':0,'Yes':1}},inplace=True)
# label encoding
df.replace({"Education":{'Not Graduate':0,'Graduate':1}},inplace=True)
# label encoding
df.replace({"Self_Employed":{'No':0,'Yes':1}},inplace=True)
# label encoding
df.replace({"Loan_Status":{'N':0,'Y':1}},inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_12896\2835109398.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"Gender":{'Female':0,'Male':1}},inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_12896\2835109398.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"Married":{'No':0,'Yes':1}},inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_12896\2835109398.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, ex

In [29]:
df['Gender'] = df['Gender'].astype(int)
df['Married'] = df['Married'].astype(int)
df['Education'] = df['Education'].astype(int)
df['Self_Employed'] = df['Self_Employed'].astype(int)

df['CoapplicantIncome'] = df['CoapplicantIncome'].astype(int)
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].astype(int)
df['LoanAmount'] = df['LoanAmount'].astype(int)

In [30]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,1,0,5849,0,146,360,1.0,Urban,1
1,LP001003,1,1,1,1,0,4583,1508,128,360,1.0,Rural,0
2,LP001005,1,1,0,1,1,3000,0,66,360,1.0,Urban,1
3,LP001006,1,1,0,0,0,2583,2358,120,360,1.0,Urban,1
4,LP001008,1,0,0,1,0,6000,0,141,360,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0,1,0,2900,0,71,360,1.0,Rural,1
610,LP002979,1,1,3+,1,0,4106,0,40,180,1.0,Rural,1
611,LP002983,1,1,1,1,0,8072,240,253,360,1.0,Urban,1
612,LP002984,1,1,2,1,0,7583,0,187,360,1.0,Urban,1


In [31]:
df['Dependents'] = df['Dependents'].replace('3+' , '4')
df['Dependents'] = df['Dependents'].astype(int)

In [32]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,1,0,5849,0,146,360,1.0,Urban,1
1,LP001003,1,1,1,1,0,4583,1508,128,360,1.0,Rural,0
2,LP001005,1,1,0,1,1,3000,0,66,360,1.0,Urban,1
3,LP001006,1,1,0,0,0,2583,2358,120,360,1.0,Urban,1
4,LP001008,1,0,0,1,0,6000,0,141,360,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0,1,0,2900,0,71,360,1.0,Rural,1
610,LP002979,1,1,4,1,0,4106,0,40,180,1.0,Rural,1
611,LP002983,1,1,1,1,0,8072,240,253,360,1.0,Urban,1
612,LP002984,1,1,2,1,0,7583,0,187,360,1.0,Urban,1


In [33]:
df.dtypes

Loan_ID               object
Gender                 int32
Married                int32
Dependents             int32
Education              int32
Self_Employed          int32
ApplicantIncome        int64
CoapplicantIncome      int32
LoanAmount             int32
Loan_Amount_Term       int32
Credit_History       float64
Property_Area         object
Loan_Status            int64
dtype: object

In [34]:
df['Credit_History'] = df['Credit_History'].fillna(1.0)

In [35]:
df['Credit_History'] = df['Credit_History'].astype(int)

In [36]:
Loan_Idd = df['Loan_ID']

In [37]:
df = df.drop(columns=['Loan_ID'])

In [38]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,1,0,5849,0,146,360,1,Urban,1
1,1,1,1,1,0,4583,1508,128,360,1,Rural,0
2,1,1,0,1,1,3000,0,66,360,1,Urban,1
3,1,1,0,0,0,2583,2358,120,360,1,Urban,1
4,1,0,0,1,0,6000,0,141,360,1,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,1,0,2900,0,71,360,1,Rural,1
610,1,1,4,1,0,4106,0,40,180,1,Rural,1
611,1,1,1,1,0,8072,240,253,360,1,Urban,1
612,1,1,2,1,0,7583,0,187,360,1,Urban,1


In [39]:
df['Property_Area'].value_counts()

Property_Area
Semiurban    233
Urban        202
Rural        179
Name: count, dtype: int64

In [40]:

df.replace({"Property_Area":{'Semiurban':1,'Urban':2,'Rural':3}},inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_12896\4130514100.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"Property_Area":{'Semiurban':1,'Urban':2,'Rural':3}},inplace=True)


In [41]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,1,0,5849,0,146,360,1,2,1
1,1,1,1,1,0,4583,1508,128,360,1,3,0
2,1,1,0,1,1,3000,0,66,360,1,2,1
3,1,1,0,0,0,2583,2358,120,360,1,2,1
4,1,0,0,1,0,6000,0,141,360,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,1,0,2900,0,71,360,1,3,1
610,1,1,4,1,0,4106,0,40,180,1,3,1
611,1,1,1,1,0,8072,240,253,360,1,2,1
612,1,1,2,1,0,7583,0,187,360,1,2,1


In [42]:
df.dtypes

Gender               int32
Married              int32
Dependents           int32
Education            int32
Self_Employed        int32
ApplicantIncome      int64
CoapplicantIncome    int32
LoanAmount           int32
Loan_Amount_Term     int32
Credit_History       int32
Property_Area        int64
Loan_Status          int64
dtype: object

In [120]:
X = df.drop('Loan_Status' , axis=1)
Y = df['Loan_Status']

X_train , X_test , y_train , y_test = train_test_split(X , Y , test_size=.1, random_state=3)
X_train.shape , X_test.shape , y_train.shape , y_test.shape 


((552, 11), (62, 11), (552,), (62,))

In [121]:
model =  LogisticRegression()

In [122]:
model.fit(X_train , y_train)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [123]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [124]:
Training_Accuracy = accuracy_score(y_train_pred , y_train)
Test_Accuracy = accuracy_score(y_test_pred , y_test)

In [125]:
print(f'Training Data Accuracy :  {Training_Accuracy*100} %')
print(f'Test Data Accuracy :  {Test_Accuracy*100} %')

Training Data Accuracy :  80.43478260869566 %
Test Data Accuracy :  87.09677419354838 %
